In [2]:
%pylab inline 
import pandas as pd
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [3]:
prob = np.load("prob.npy")
prob

array([0.98828207, 0.986881  , 0.98568157, 0.98477179, 0.98408444,
       0.98346398, 0.98280699, 0.98214304, 0.98144266, 0.98070591,
       0.97995497, 0.97921927, 0.97847673, 0.97772003, 0.97696388,
       0.97609828, 0.97513816, 0.97418624, 0.97324249, 0.97224116,
       0.97106564, 0.96963635, 0.96793982, 0.96591924, 0.96354757,
       0.9608778 , 0.95785464, 0.95435201, 0.95031767, 0.94577225,
       0.94083612, 0.93553634, 0.9297738 , 0.92353596, 0.91685984,
       0.90969997, 0.90218345, 0.89454917, 0.88689476, 0.87906873,
       0.87050953, 0.86129715, 0.85202726, 0.84281219, 0.83325994,
       0.82248918, 0.8102755 , 0.79702273, 0.78268864, 0.76694275,
       0.74895781, 0.72875588, 0.70698773, 0.68374187, 0.6586057 ,
       0.6303344 , 0.59924801, 0.56669902, 0.53306456, 0.49787087,
       0.45973063, 0.41937498, 0.37879578, 0.33867486, 0.29885913,
       0.25929479, 0.22041799, 0.18307063, 0.14827566, 0.11698081,
       0.        ])

In [3]:
det = np.load("detEarning.npy")
det

array([ 35.        ,  37.16666667,  39.33333333,  41.5       ,
        43.66666667,  45.83333333,  48.        ,  50.16666667,
        52.33333333,  54.5       ,  56.66666667,  58.83333333,
        61.        ,  63.16666667,  65.33333333,  67.5       ,
        69.66666667,  71.83333333,  74.        ,  76.16666667,
        78.33333333,  80.5       ,  82.66666667,  84.83333333,
        87.        ,  89.16666667,  91.33333333,  93.5       ,
        95.66666667,  97.83333333, 100.        ,  97.        ,
        94.        ,  91.        ,  88.        ,  85.        ,
        82.        ,  79.        ,  76.        ,  73.        ,
        70.        ,  63.        ,  56.        ,  49.        ,
        42.        ,  35.        ,  35.        ,  35.        ,
        35.        ,  35.        ,  35.        ,  35.        ,
        35.        ,  35.        ,  35.        ,  35.        ,
        35.        ,  35.        ,  35.        ,  35.        ,
        35.        ,  35.        ,  35.        ,  35.  

In [5]:
sum([det[i]*((1+0.02)**(30-i)) for i in range(30)])

2616.0531797955164

In [1]:
35*(1.2**5)

87.09119999999999

In [7]:
len(det)

71

In [13]:
sum(det[:45])*0.05

156.875

In [12]:
det

array([ 35.        ,  37.16666667,  39.33333333,  41.5       ,
        43.66666667,  45.83333333,  48.        ,  50.16666667,
        52.33333333,  54.5       ,  56.66666667,  58.83333333,
        61.        ,  63.16666667,  65.33333333,  67.5       ,
        69.66666667,  71.83333333,  74.        ,  76.16666667,
        78.33333333,  80.5       ,  82.66666667,  84.83333333,
        87.        ,  89.16666667,  91.33333333,  93.5       ,
        95.66666667,  97.83333333, 100.        ,  97.        ,
        94.        ,  91.        ,  88.        ,  85.        ,
        82.        ,  79.        ,  76.        ,  73.        ,
        70.        ,  63.        ,  56.        ,  49.        ,
        42.        ,  35.        ,  35.        ,  35.        ,
        35.        ,  35.        ,  35.        ,  35.        ,
        35.        ,  35.        ,  35.        ,  35.        ,
        35.        ,  35.        ,  35.        ,  35.        ,
        35.        ,  35.        ,  35.        ,  35.  

In [2]:
import random

In [10]:
%pylab inline
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from multiprocessing import Pool
from functools import partial 
from pyswarm import pso
import warnings
warnings.filterwarnings("ignore")
np.printoptions(precision=2)
# time line
T_min = 0
T_max = 70
T_R = 45
beta = 1/(1+0.02)
# All the money amount are denoted in thousand dollars
earningShock = [0.8,1.2]
# Define transition matrix of economical states
# GOOD -> GOOD 0.8, BAD -> BAD 0.6 
Ps = np.array([[0.6, 0.4],[0.2, 0.8]])
# current risk free interest rate 
r_f = np.array([0.01 ,0.03])
# stock return depends on current and future econ states
r_m = np.array([[-0.2, 0.15],[-0.15, 0.2]])
# probability of survival
Pa = np.load("prob.npy")
# probability of employment transition 
Pe = np.array([[[[0.3, 0.7], [0.1, 0.9]], [[0.25, 0.75], [0.05, 0.95]]],
               [[[0.25, 0.75], [0.05, 0.95]], [[0.2, 0.8], [0.01, 0.99]]]])
# deterministic income
detEarning = np.load("detEarning.npy")
# tax rate 
tau_L = 0.2
tau_R = 0.1
# minimum consumption
c_bar = 3

Populating the interactive namespace from numpy and matplotlib


In [18]:
t = 30
r_bar = 0.0667
N = int(np.sum(Pa[t:]))
discouting = ((1+r_bar)**N - 1)/(((1+r_bar)**N - (1+r_bar)**(N-1)))
discouting

13.008380275151591

In [8]:
# Define the transtiion of state 
def transition(x, a, t, s_next):
    '''
        Input: x current state: (w, n, s, A) 
               a action taken: (c, b, k)
        Output: the next possible states with corresponding probabilities
    '''
    c, b, k = a
    w, s, e, A = x
    
    x_next = []
    prob_next = []
    if A == 0:
        for s_next in [0, 1]:
            x_next.append([0, s_next, 0, 0])
        return np.array(x_next), Ps[int(s)]
    else:
        # A = 1, agent is still alive and for the next period
        Pat = [1-Pa[t], Pa[t]]
        r_bond = r_f[int(s)]
        r_stock = r_m[int(s), s_next]
        w_next =  b*(1+r_bond) + k*(1+r_stock)
        for e_next in [0,1]:
            for A_next in [0,1]:
                x_next.append([w_next, s_next, e_next, A_next])
                prob_next.append(Pat[A_next] * Pe[int(s),s_next,int(e),e_next])
    return x_next[np.random.choice(4, 1, p = prob_next)]

In [166]:
x_possible_next, next_prob = transition([100,0,0,1], [10,10,10], 20, 0)

In [235]:
x_possible_next[(np.random.choice(4, 1, p=next_prob))]

array([[18.1,  0. ,  1. ,  1. ]])

In [3]:
import numpy as np

In [14]:
ww = [1,2,3,4]
a = np.array([[w,s,e,A]for w in ww for s in [0, 1] for e in [0,1] for A in [0,1]])
a.reshape((4,2,2,2,4))

array([[[[[1, 0, 0, 0],
          [1, 0, 0, 1]],

         [[1, 0, 1, 0],
          [1, 0, 1, 1]]],


        [[[1, 1, 0, 0],
          [1, 1, 0, 1]],

         [[1, 1, 1, 0],
          [1, 1, 1, 1]]]],



       [[[[2, 0, 0, 0],
          [2, 0, 0, 1]],

         [[2, 0, 1, 0],
          [2, 0, 1, 1]]],


        [[[2, 1, 0, 0],
          [2, 1, 0, 1]],

         [[2, 1, 1, 0],
          [2, 1, 1, 1]]]],



       [[[[3, 0, 0, 0],
          [3, 0, 0, 1]],

         [[3, 0, 1, 0],
          [3, 0, 1, 1]]],


        [[[3, 1, 0, 0],
          [3, 1, 0, 1]],

         [[3, 1, 1, 0],
          [3, 1, 1, 1]]]],



       [[[[4, 0, 0, 0],
          [4, 0, 0, 1]],

         [[4, 0, 1, 0],
          [4, 0, 1, 1]]],


        [[[4, 1, 0, 0],
          [4, 1, 0, 1]],

         [[4, 1, 1, 0],
          [4, 1, 1, 1]]]]])

In [15]:
a.sum(axis = 1).reshape(4,2,2,2)

array([[[[1, 2],
         [2, 3]],

        [[2, 3],
         [3, 4]]],


       [[[2, 3],
         [3, 4]],

        [[3, 4],
         [4, 5]]],


       [[[3, 4],
         [4, 5]],

        [[4, 5],
         [5, 6]]],


       [[[4, 5],
         [5, 6]],

        [[5, 6],
         [6, 7]]]])

In [ ]:
a.sum(axis = 1).reshape((4,2))

In [15]:
def demo_func(x):
    x1, x2, x3 = x
    return x1 ** 2 + (x2 - 0.05) ** 2 + x3 ** 2

In [24]:
from sko.PSO import PSO

pso = PSO(func=demo_func, dim=3, pop=40, max_iter=150, lb=[0, -1, 0.5], ub=[1, 1, 1], w=0.8, c1=0.5, c2=0.5)
pso.run()
print('best_x is ', pso.gbest_x, 'best_y is', pso.gbest_y)

best_x is  [0.   0.05 0.5 ] best_y is 0.25


In [21]:
pso.gbest_x

array([0.  , 0.05, 0.5 ])

In [14]:
pso.gbest_y

0.25